In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import src.config as config 

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1018005
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (71.89s) 
2024-09-11 13:06:46,174 WARNING: VersionWarning: Incremented version to `16`.



In [6]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data


,pickup_hour,rides,pickup_location_id
1191329,2022-01-01 00:00:00+00:00,0,1
3839458,2022-01-01 01:00:00+00:00,0,1
1159772,2022-01-01 02:00:00+00:00,0,1
1340779,2022-01-01 03:00:00+00:00,0,1
3501991,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
3715036,2024-09-11 02:00:00+00:00,3,265
3715411,2024-09-11 03:00:00+00:00,1,265
3715546,2024-09-11 04:00:00+00:00,2,265
3715846,2024-09-11 05:00:00+00:00,1,265


In [7]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

2024-09-11 13:06:54,645 WARNING: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)

2024-09-11 13:06:54,861 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [8]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

  0%|          | 0/257 [00:00<?, ?it/s]

2024-09-11 13:06:56,143 WARNING: d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)




  0%|          | 1/257 [00:00<01:25,  3.00it/s]

2024-09-11 13:06:56,410 WARNING: d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


2024-09-11 13:06:56,477 WARNING: d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)




  1%|          | 3/257 [00:00<00:56,  4.50it/s]

...

2024-09-11 13:09:19,581 WARNING: d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)




100%|█████████▉| 256/257 [02:24<00:00,  1.47it/s]

2024-09-11 13:09:20,300 WARNING: d:\Real_world_Ml_Project\taxi_demand_predictor\.venv\lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)




100%|██████████| 257/257 [02:24<00:00,  1.77it/s]


features_and_target.shape=(218242, 675)


In [9]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29 00:00:00+00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-01-29 23:00:00+00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,2.0,2.0,0.0,1.0,2.0,0.0,0.0,2022-01-30 22:00:00+00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,0.0,1.0,2022-01-31 21:00:00+00:00,1,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-02-01 20:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218237,2.0,1.0,2.0,0.0,2.0,7.0,6.0,1.0,2.0,3.0,...,5.0,11.0,4.0,2.0,7.0,12.0,8.0,2024-09-07 01:00:00+00:00,265,7.0
218238,5.0,10.0,4.0,2.0,6.0,0.0,3.0,4.0,4.0,2.0,...,6.0,1.0,5.0,4.0,7.0,2.0,10.0,2024-09-08 00:00:00+00:00,265,3.0
218239,3.0,7.0,4.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,3.0,8.0,5.0,13.0,6.0,9.0,2024-09-08 23:00:00+00:00,265,9.0
218240,10.0,2.0,7.0,2.0,2.0,1.0,0.0,0.0,3.0,1.0,...,7.0,4.0,9.0,6.0,9.0,4.0,7.0,2024-09-09 22:00:00+00:00,265,9.0


In [11]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

features_and_target["pickup_hour"] = pd.to_datetime(features_and_target["pickup_hour"])

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-08-14 00:00:00+0000', tz='UTC')
X_train.shape=(211288, 674)
y_train.shape=(211288,)
X_test.shape=(6954, 674)
y_test.shape=(6954,)


In [12]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
         
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2024-09-11 13:34:38,385] A new study created in memory with name: no-name-30ca416f-e703-49f4-9678-5c368ea2db90


2024-09-11 13:34:39,485 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:35:32,449 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:35:35,403 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

[I 2024-09-11 13:36:57,723] Trial 0 finished with value: 3.1861164859688054 and parameters: {'num_leaves': 11, 'feature_fraction': 0.5850952802611824, 'bagging_fraction': 0.8475417788882029, 'min_child_samples': 63}. Best is trial 0 with value: 3.1861164859688054.


2024-09-11 13:36:59,164 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:37:45,316 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:37:48,368 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

[I 2024-09-11 13:39:24,458] Trial 1 finished with value: 3.2539608012912358 and parameters: {'num_leaves': 9, 'feature_fraction': 0.5289988190195701, 'bagging_fraction': 0.6306236514122812, 'min_child_samples': 35}. Best is trial 0 with value: 3.1861164859688054.


...

[I 2024-09-11 13:52:36,036] Trial 7 finished with value: 3.062675815496005 and parameters: {'num_leaves': 41, 'feature_fraction': 0.6675993322343923, 'bagging_fraction': 0.35425290682201827, 'min_child_samples': 60}. Best is trial 2 with value: 2.994992547088735.


2024-09-11 13:52:36,521 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:53:02,185 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:53:03,552 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

[I 2024-09-11 13:53:54,254] Trial 8 finished with value: 3.086618952220077 and parameters: {'num_leaves': 28, 'feature_fraction': 0.6053689956690944, 'bagging_fraction': 0.885467589273409, 'min_child_samples': 11}. Best is trial 2 with value: 2.994992547088735.


2024-09-11 13:53:54,708 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:54:40,548 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2024-09-11 13:54:42,261 WARNING: D:\Real_world_Ml_Project\taxi_demand_predictor\src\model.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

[I 2024-09-11 13:56:16,144] Trial 9 finished with value: 3.0041916404493727 and parameters: {'num_leaves': 134, 'feature_fraction': 0.4009025199752226, 'bagging_fraction': 0.3282334453944086, 'min_child_samples': 94}. Best is trial 2 with value: 2.994992547088735.


In [14]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 150, 'feature_fraction': 0.4576721724346207, 'bagging_fraction': 0.990878647061735, 'min_child_samples': 45}


In [15]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.4576721724346207, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576721724346207
[LightGBM] [Warning] bagging_fraction is set=0.990878647061735, subsample=1.0 will be ignored. Current value: bagging_fraction=0.990878647061735
[LightGBM] [Warning] feature_fraction is set=0.4576721724346207, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576721724346207
[LightGBM] [Warning] bagging_fraction is set=0.990878647061735, subsample=1.0 will be ignored. Current value: bagging_fraction=0.990878647061735
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171876
[LightGBM] [Info] Number of data points in the train set: 211288, number of used features: 676
[LightGBM] [Info] Start training from score 1

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000024A25013820>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.990878647061735,
                               feature_fraction=0.4576721724346207,
                               min_child_samples=45, num_leaves=150))])

In [16]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.4576721724346207, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4576721724346207
[LightGBM] [Warning] bagging_fraction is set=0.990878647061735, subsample=1.0 will be ignored. Current value: bagging_fraction=0.990878647061735
test_mae=4.7215


In [17]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['D:\\Real_world_Ml_Project\\taxi_demand_predictor\\models\\model.pkl']

In [18]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [19]:
model_registry = project.get_model_registry()


model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR / 'model.pkl'))

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1362610 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3404 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60849 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1018005/models/taxi_demand_predictor_next_hour/2


Model(name: 'taxi_demand_predictor_next_hour', version: 2)